# Find Stocks with Fluctuation Suitable for Covered Call


In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime as dt
import pandas_datareader.data as web
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
import yfinance as yf
import bs4 as bs
import pickle
import requests
import sys
import os
from tqdm import tqdm # progress

THIS_FOLDER = os.path.dirname(os.path.abspath(__file__))

style.use('ggplot')
mpl.rcParams['figure.figsize'] = (16.0, 9.0)
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'

data_path = '../data/all_stock'

def prev_weekday(adate):
    # adate -= dt.timedelta(days=1)
    while adate.weekday() > 4: # Mon-Fri are 0-4
        adate -= dt.timedelta(days=1)
    return adate

def next_friday(adate):
    adate += dt.timedelta(days=1)
    while adate.weekday() != 4: # Mon-Fri are 0-4
        adate += dt.timedelta(days=1)
    return adate

date_2009low = prev_weekday(dt.datetime(2009, 3, 9))

date_start = prev_weekday(dt.datetime(2015, 1, 6))
date2 = dt.datetime(2020, 2, 19)
date_end = prev_weekday(dt.datetime.now())

print('Lasted weekday:', date_end)
      
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.rstrip().replace('.', '-'))
        
#     with open("sp500tickers.pickle","wb") as f:
#         pickle.dump(tickers,f)
        
    print('Total number: ', len(tickers))
    return tickers
            
tickers = save_sp500_tickers()[:]
today_price = []

C:\Users\lfu7\AppData\Roaming\Python\Python37\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


Lasted weekday: 2020-06-26 21:13:46.474534
Total number:  505


In [2]:
def compile_data(tickers = tickers):

    dDay = 7
    anomaly = []
    main_df = pd.DataFrame()
    deltaWeek = dt.timedelta(days=7)
    shiftYear = dt.timedelta(days=365)

    for count, ticker in enumerate(tqdm(tickers[:])):
        print(count, ': ', ticker)
        
        ticker = ticker.lstrip().rstrip().replace('~', '').replace('$', '')
        
        useLocalData = os.path.exists(data_path+'/{}.csv'.format(ticker))

        if useLocalData:
#             print('Retrieve data from '+data_path+'/{}.csv'.format(ticker))
            df = pd.read_csv(data_path+'/{}.csv'.format(ticker))
        else:
            print(ticker, ' does not exist! Skip...')
            anomaly.append(ticker)
            continue
        
        df.set_index("Date", inplace=True)
        df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    
        num_weeks = 4
        perc_nw = np.full((len(df.index), num_weeks), np.nan)
#         print(perc_nw)
#         print(len(perc_nw), len(perc_nw[0]))
 
        for i, day in enumerate(df.index[:]):
        
            perc = []
            
            for j in range(num_weeks):
                # find Friday j weeks later
                friday = next_friday(day + j * deltaWeek)
                           
                # find the expiration date j weeks later
                nweek = df.loc[:friday, :].tail(1).index

#                 print(day + j * deltaWeek, friday, nweek[-1])
            
                perc = df.loc[nweek, 'Adj Close'] / df.loc[day, 'Adj Close'] - 1.0
                perc_nw[i][j] = perc.values[0] 
#                 print(i, j, perc_nw[i][j])

                
#             df2 = pd.concat([x for x in perc])
#             df2..rename(columns={'Adj Close': ticker}, inplace=True)
#             print('df2', df2, type(df2))

        df_perc = pd.DataFrame(data=perc_nw, index=df.index, columns=[str(i+1) + ' week' for i in range(num_weeks)])

#         print(df.head(), df.describe())
        data = pd.concat([df, df_perc], axis=1)
#         print(df_perc.head(), df_perc.tail(), df_perc.describe())
#         print(data.info(), data.describe(), data.head(), data.tail())
        
        fullname = os.path.join('./data/', '{}.csv'.format(ticker))
        data.to_csv(fullname)
        
        
    return anomaly
        

anomaly = compile_data()
print('Number of anomalous tickers:', len(anomaly))
if len(anomaly):
    print('Anomalous tickers:', anomaly)

  0%|                                                                                          | 0/505 [00:00<?, ?it/s]

0 :  MMM


  0%|▏                                                                               | 1/505 [00:16<2:19:48, 16.64s/it]

1 :  ABT


  0%|▎                                                                               | 2/505 [00:33<2:20:06, 16.71s/it]

...

503 :  ZION


100%|█████████████████████████████████████████████████████████████████████████████▊| 504/505 [2:02:12<00:15, 15.34s/it]

504 :  ZTS


100%|██████████████████████████████████████████████████████████████████████████████| 505/505 [2:02:17<00:00, 14.53s/it]

Number of anomalous tickers: 4
Anomalous tickers: ['BRK-B', 'BF-B', 'IPGP', 'SBUX']
